We examine the predictions of normally trained classifiers for adversarial perturbations on natural samples or uniform noises created by `create.py`. Specifically, we assess whether:
- The adversarial attacks we implemented work as intended.
- The adversarial datasets in each scenario successfully mislead the classifiers.
    - Notably, certain adversarial images failed to deceive the classifiers, possibly because of learning bias, architectural bias, or suboptimal PGD optimization.

For example, we examined the following values.

Dataset: CIFAR10  
Scenario: natural_det_L2  
Accuracy: [0.98 (1), 0.99 (2), ..., 0.99 (10)]

(1) Ratio of truck images (with imperceptible L2 perturbations to mislead the classifier into identifying them as planes) classified as planes by the classifier.  
(2) Ratio of plane images (with imperceptible L2 perturbations to mislead the classifier into identifying them as cars) classified as cars by the classifier.  
(10) Ratio of ship images (with imperceptible L2 perturbations to mislead the classifier into identifying them as trucks) classified as trucks by the classifier.  

Dataset: CIFAR10  
Scenario: natural_rand_L2  
Accuracy: [0.99 (1), ...]

(1) Ratio of images that appear as objects other than planes (with imperceptible L2 perturbations to mislead the classifier into identifying them as planes) classified as planes by the classifier.  

These results indicate that L2 perturbations on natural samples can effectively fool the classifier.

Dataset: CIFAR10  
Scenario: uniform_L2  
Accuracy: [0.07 (1), 0.00, 0.00, 1.0 (4), ...]

(1) Ratio of noises (with imperceptible L2 perturbations to mislead the classifier into identifying them as planes) classified as planes by the classifier.  
(4) Ratio of noises (with imperceptible L2 perturbations to mislead the classifier into identifying them as cats) classified as cats by the classifier.  

This suggests that while L2 perturbations on noises can successfully mislead the classifier to identify them as cats, they are less effective in doing so for planes.

In [1]:
import os
root = os.path.join('..', '..')

In [2]:
import sys
sys.path.append(root)

In [3]:
from collections import OrderedDict
from typing import Literal

import torch

from utils.classifiers import ConvNet, WideResNet
from utils.datasets import CIFAR10, FMNIST, MNIST, SequenceDataset
from utils.utils import (CalcClassificationAcc, ModelWithNormalization,
                         dataloader, freeze)

In [4]:
device = [0]
dataset_root = os.path.join(os.path.sep, 'root', 'datasets')

In [5]:
class Util:
    def __init__(self, dataset_name: Literal['MNIST', 'FMNIST', 'CIFAR10']) -> None:
        self.dataset_name = dataset_name
        self.classifier = self._define_classifier()
        self._load_weight()

    def _define_classifier(self) -> ModelWithNormalization:
        if self.dataset_name == 'MNIST':
            classifier = ConvNet(10)
            dataset_cls = MNIST
        elif self.dataset_name == 'FMNIST':
            classifier = ConvNet(10)
            dataset_cls = FMNIST
        elif self.dataset_name == 'CIFAR10':
            classifier = WideResNet(28, 10, 0.3, 10)
            dataset_cls = CIFAR10
        else:
            raise ValueError(self.dataset_name)
        return ModelWithNormalization(classifier, dataset_cls.mean, dataset_cls.std)

    def _load_weight(self) -> None:
        dir_path = os.path.join(root, 'models', self.dataset_name, 'version_0', 'checkpoints')
        ckpt_name = [fname for fname in os.listdir(dir_path) if '.ckpt' in fname][0]
        path = os.path.join(dir_path, ckpt_name)

        state_dict = torch.load(path, map_location='cpu')['state_dict']
        state_dict = OrderedDict((k.replace('classifier.', ''), v) for k, v in state_dict.items())
        self.classifier.load_state_dict(state_dict)

        freeze(self.classifier)
        self.classifier.eval()

    def _load_dataset(self, suffix: str) -> SequenceDataset:
        p = os.path.join(root, 'datasets', f'{self.dataset_name}_{suffix}', 'dataset')
        d = torch.load(p, map_location='cpu')
        return SequenceDataset(d['imgs'], d['labels'])
    
    def test(self, suffix: str) -> None:
        print(suffix)

        if self.dataset_name in ('MNIST', 'FMNIST'):
            batch_size = 60000
        elif self.dataset_name == 'CIFAR10':
            batch_size = 10000
        else:
            raise ValueError(self.dataset_name)

        d = self._load_dataset(suffix)
        loader = dataloader(d, batch_size, False)

        acc = CalcClassificationAcc(
            accelerator='gpu',
            strategy='dp',
            devices=device,
            precision=16,
        ).run(self.classifier, loader, 10, average='none')
        print(acc)
        print()
    
    def test_all(self) -> None:
        names = [
            'natural_rand_L0', 
            'natural_det_L0',
            'natural_rand_L2', 
            'natural_det_L2',
            'natural_rand_Linf', 
            'natural_det_Linf',
            'uniform_L0', 
            'uniform_L2', 
            'uniform_Linf', 
        ]
        for n in names:
            self.test(n)

In [6]:
Util('MNIST').test_all()

Using 16-bit Automatic Mixed Precision (AMP)


natural_rand_L0


100%|██████████| 1/1 [00:02<00:00,  2.27s/it]


[0.013366336934268475, 0.025553662329912186, 0.02901674620807171, 0.057287417352199554, 0.04490472376346588, 0.06073211133480072, 0.026341622695326805, 0.05843157321214676, 0.06939322501420975, 0.07564016431570053]

natural_det_L0


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 1/1 [00:01<00:00,  1.39s/it]


...

[0.0, 0.0, 2.5024606657098047e-05, 0.008678583428263664, 0.002002193359658122, 0.0006218957132659853, 0.0, 0.0005088676698505878, 1.0, 0.0]

uniform_L2


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 20/20 [00:08<00:00,  2.34it/s]


[0.00029195132083259523, 0.0005647090147249401, 0.9998582005500793, 0.9999916553497314, 1.0, 0.9999419450759888, 0.9699679017066956, 0.9999666213989258, 1.0, 0.01938806101679802]

uniform_Linf


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 20/20 [00:08<00:00,  2.38it/s]


[0.05436967685818672, 0.01022289413958788, 0.990340530872345, 0.9962663054466248, 0.9986790418624878, 0.9742452502250671, 0.9347004294395447, 0.9922835826873779, 1.0, 0.014872805215418339]



In [7]:
Util('FMNIST').test_all()

natural_rand_L0


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 1/1 [00:01<00:00,  1.22s/it]


[0.3781554102897644, 0.292400598526001, 0.41621533036231995, 0.26822832226753235, 0.5483601093292236, 0.10065645724534988, 0.545543909072876, 0.14194950461387634, 0.4046885669231415, 0.022753890603780746]

natural_det_L0


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 1/1 [00:01<00:00,  1.23s/it]


[0.4749999940395355, 0.48233333230018616, 0.2018333375453949, 0.33783334493637085, 0.6633333563804626, 0.026000000536441803, 0.4908333420753479, 0.009333333000540733, 0.5958333611488342, 0.005166666582226753]

natural_rand_L2


...

[0.9793548583984375, 0.9898186326026917, 0.9675514101982117, 0.9989141225814819, 0.9999584555625916, 0.3508155047893524, 1.0, 0.00011678929877234623, 1.0, 0.006242905743420124]

uniform_L2


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 20/20 [00:08<00:00,  2.28it/s]


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9785107970237732, 1.0, 0.9991152882575989]

uniform_Linf


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 20/20 [00:08<00:00,  2.34it/s]


[1.0, 0.999717652797699, 1.0, 1.0, 1.0, 0.9807378053665161, 1.0, 0.9213257431983948, 1.0, 0.8664702773094177]



In [8]:
Util('CIFAR10').test_all()

natural_rand_L0


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 5/5 [02:43<00:00, 32.75s/it]


[0.988476037979126, 0.9900099635124207, 0.9836294651031494, 0.981062114238739, 0.9810436367988586, 0.971230149269104, 0.9867036938667297, 0.9698060154914856, 0.9761021733283997, 0.9917852282524109]

natural_det_L0


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 5/5 [02:43<00:00, 32.68s/it]


[0.9818000197410583, 0.9715999960899353, 0.9377999901771545, 0.9954000115394592, 0.9927999973297119, 0.9959999918937683, 0.995199978351593, 0.9656000137329102, 0.9890000224113464, 0.993399977684021]

natural_rand_L2


...

[1.0, 0.9836890697479248, 0.9325553774833679, 1.0, 0.9884240627288818, 0.8388190865516663, 1.0, 0.3525976538658142, 0.14874586462974548, 1.0]

uniform_L2


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 5/5 [02:43<00:00, 32.63s/it]


[0.07638049125671387, 0.0, 0.0, 1.0, 0.0, 0.0, 0.9187073707580566, 0.0, 0.0, 0.9943752288818359]

uniform_Linf


Using 16-bit Automatic Mixed Precision (AMP)
100%|██████████| 5/5 [01:35<00:00, 19.04s/it]

[0.8597339987754822, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.9995982050895691]

